In [11]:
import torch as t
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import SGDRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import StandardScaler

In [ ]:
dataset = t.load("data/ffa_channelpark.pt", weights_only=False)
observations,_ = dataset.tensors
positions = observations[:,:3].float()

angles =  10 * positions[:, 2] * t.pi / 180
cosi = t.stack([t.cos(angles), t.sin(angles)], dim=1)
positions = t.cat((positions[:, :2], cosi,positions[:,3:]), dim=1)

scaler = StandardScaler()
positions_scaled = t.tensor(scaler.fit_transform(positions))

X = positions[:-1].unfold(dimension=0, size=10, step=1).flatten(start_dim=1,end_dim=2)
y =  positions[10:]  

X_scaled = positions_scaled[:-1].unfold(dimension=0, size=10, step=1).flatten(start_dim=1,end_dim=2)
y_scaled =  positions_scaled[10:]  

print("Data shape:\t",positions.shape)
print("Input shape:\t",X.shape)
print("Output shape:\t",y.shape)

AttributeError: 'numpy.ndarray' object has no attribute 'unfold'

## Constant model

In [ ]:
mse  = mean_squared_error(positions[1:], positions[:-1])
print("Mean squared error:\t",mse)

Mean squared error:	 0.00035304092751520277


##  Linear Regression

In [ ]:
reg = LinearRegression().fit(X, y)
y_pred = reg.predict(X)
mse  = mean_squared_error(y, y_pred)
print("Mean squared error:\t",mse)

Mean squared error:	 0.0017755660228431225


## Multioutput Regression using SGD

In [ ]:
model = MultiOutputRegressor(SGDRegressor(loss="squared_error",tol=1e-2)).fit(X, y)
y_pred = model.predict(X)
mse  = mean_squared_error(y, y_pred)
print("Mean squared error:\t",mse)

Mean squared error:	 0.00037190102697359377


In [ ]:
model = MultiOutputRegressor(SGDRegressor(loss="squared_error",tol=1e-2)).fit(X_scaled, y_scaled)
y_pred = model.predict(X_scaled)
mse  = mean_squared_error(y_scaled, y_pred)
print("Mean squared error:\t",mse)



NameError: name 'y_scaled' is not defined